# PSTAT 134
## J Steven Raquel
## Fri 9am Section


### Assignment 3

Our objective is to understand non-negative matrix factorization (NMF) and analyze more basketball player data. 

NMF was used on the shooting pattern data for 362 players. It has two components: the _base_ which condenses down all of the players' unique charts into a number of modes of shooting styles, which we can determine with the `n_components` argument in the `NMF` function, and the _coefficients_ which expresses how each of the players shooting styles can be expressed as a linear combination of the bases. 

In [ ]:
# setting up the environment
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import subprocess as sp
import pickle
import sklearn.decomposition as skld

import helper_basketball as h
import imp
imp.reload(h);

In [ ]:
# get all 2016-17 shots
allshots = pickle.load(open('allshots2016-17.pkl', 'rb'))
allmade = allshots.loc[allshots.SHOT_MADE_FLAG==1]
allmade = allshots
## get all 2016-17 teams
params = {'LeagueID':'00','Season': '2016-17'}
teams = h.get_nba_data('commonTeamYears', params).set_index('TEAM_ID')
allteams = teams.loc[teams.MAX_YEAR=='2017'].index.values
## get all 2016-17 players
params = {'LeagueID':'00', 'Season': '2016-17', 'IsOnlyCurrentSeason': '0'}
players = h.get_nba_data('commonallplayers', params).set_index('PERSON_ID')
allplyrs = players.loc[players.TEAM_ID.isin(allteams)].index.values

In [ ]:
## players info
player_ids = allmade.PlayerID.unique()
num_players = player_ids.size

## bin edge definitions in inches
xedges = (np.linspace(start=-25, stop=25, num=151, dtype=np.float)) * 12
yedges = (np.linspace(start= -4, stop=31, num=106, dtype=np.float)) * 12

## number of bins is one less than number of edges
nx = xedges.size - 1
ny = yedges.size - 1

## 2d histogram containers for binned counts and smoothed binned counts
all_counts = {}
all_smooth = {}

## data matrix: players (row) by vectorized 2-d court locations (column)
for i, one in enumerate(allmade.groupby('PlayerID')):
    
    ## what does this line do?
    pid, pdf = one
    
    ## h.bin_shots: what is this function doing?
    tmp1, xedges, yedges = h.bin_shots(pdf, bin_edges=(xedges, yedges), density=True, sigma=2)
    tmp2, xedges, yedges = h.bin_shots(pdf, bin_edges=(xedges, yedges), density=False)
    
    ## vectorize and store into dictionary
    all_smooth[pid] = tmp1.reshape(-1)
    all_counts[pid] = tmp2.reshape(-1)

### Problem 1
#### Varying the number of components

We can change the number of bases vectors with the `n_components` argument of the `NMF` function. I'll visualize a different number of bases to find out the optimal number of bases that represent all of the different kinds of shooting styles in the NBA. We'll start with $10$ components, since that is what we were given, then we can increase and decrease as necessary. Note the use of `init='nndsvda'` in the function, this indicates Nonnegative Double Singular Value Decomposition (NNDSVD) with zeroes filled with the average of X. The zeroes being filled thusly indicates that sparsity is _not_ desired. 

In [ ]:
# 10 components
model10 = skld.NMF(n_components=10, init='nndsvda', max_iter=500, random_state=0)
W = model10.fit_transform(X)
H = model10.components_

# visualizing the bases
fig, ax = plt.subplots(nrows = 5, ncols = 2, figsize=(20,40))

for i, axi in enumerate(ax.flatten()):
    h.plot_shotchart(W[:,i], xedges, yedges, ax=axi)
    axi.set_title('NMF component ' + str(i))

You might argue that some of the bases depicted here are not sufficiently distinct from one another to justify their inclusion. For example, NMF component 9 shoots mostly from an angle facing the side of the basket, with some spread activity behind the line. This is pretty similar to NMF component 5. What's more there are a variety of components that when visualized show a pattern of shooting virtually mostly underneath the hoop.

In [ ]:
# 8 components
model8 = skld.NMF(n_components=8, init='nndsvda', max_iter=500, random_state=0)
W = model8.fit_transform(X)
H = model8.components_

# visualizing the bases
fig, ax = plt.subplots(nrows = 4, ncols = 2, figsize=(20,40))

for i, axi in enumerate(ax.flatten()):
    h.plot_shotchart(W[:,i], xedges, yedges, ax=axi)
    axi.set_title('NMF component ' + str(i))
    
plt.tight_layout()

With 8 shooting styles, we still run into an overrepresentation of bases concentrated directly beneath the hoop and little elsewhere. So we can go even smaller, with 5. Logically this makes sense at this stage, since there are only 5 positions in basketball.

In [ ]:
# 5 components
model5 = skld.NMF(n_components=5, init='nndsvda', max_iter=500, random_state=0)
W = model5.fit_transform(X)
H = model5.components_

# visualizing the bases 
fig, ax = plt.subplots(nrows = 5, ncols = 1, figsize=(20,40))

for i, axi in enumerate(ax.flatten()):
    h.plot_shotchart(W[:,i], xedges, yedges, ax=axi)
    axi.set_title('NMF component ' + str(i))
    
plt.tight_layout()

At this point it seems that eliminating the extra bases that show mostly red underneath the basket simply isn't going to happen. They're still going to be viewed as distinct from one another. Rather using only 5 bases actually eliminates some of the patterns that were represented uniquely when looking at 8 or 10 bases. 

Having seen this I would opt to use only 8 bases since 5 would eliminate some important patterns, and 10 has some repetition. 

## Problem 2

It is of interest to look at the pairwise correlation between shooting patterns of different players, rather than comparing them to a set of bases. 

In [ ]:
# Getting all 362 players' shooting styles
ids = players.loc[player_ids].dropna()
ids_names = ids.reset_index()[['PERSON_ID', 'DISPLAY_FIRST_LAST']]

X = pickle.load(open('allpatterns2016-17.pkl', 'rb'))
p, n = X.shape
print('Number of bins (p)   :', p)
print('Number of players (n):', n)

Currently the matrix $X$ is oriented so that the players are on the columns. Since we want to find the correlation between the columns, this is the appropriate orientation.

In [ ]:
# convert X into a DataFrame
Xdf = pd.DataFrame(data=X, columns = ids.DISPLAY_FIRST_LAST)
# correlation matrix
R = Xdf.corr(method = 'pearson')
R.shape

We see thusly that the shape of the correlation matrix $R$ is $(362,362)$, since we are comparing each of the 362 players to one another, rather than to the bases we defined in the first problem. 

Now let's visualize the matrix $R$ with the `heatmap` function from `seaborn`.

In [ ]:
import seaborn as sns
plt.figure(figsize = (20,20))
sns.heatmap(R, xticklabels = Xdf.columns, yticklabels = Xdf.columns, square = True)

In [ ]:
s = R.unstack()
# sorting from lowest to highest
low = s.sort_values(ascending = True)
low.head()

It appears the two most dissimilar players are Mike Muscala and Will Barton, with a correlation of 0.078898. 

In [ ]:
# player IDs
muscala = ids_names.loc[ids_names['DISPLAY_FIRST_LAST'] == 'Mike Muscala', 'PERSON_ID'].item()
barton = ids_names.loc[ids_names['DISPLAY_FIRST_LAST'] == 'Will Barton', 'PERSON_ID'].item()

## create figure and axes
fig, ax = plt.subplots(2, 2, figsize=(10,20))

# 200782 = PJ Tucker, 201188 = Richard Jefferson
for axi, plyri in enumerate([muscala, barton]):
    
    h.plot_shotchart(all_counts[plyri], xedges, yedges, ax=ax[axi,0])
    h.plot_shotchart(all_smooth[plyri], xedges, yedges, ax=ax[axi,1])
    
    ax[axi,0].set_title(players.DISPLAY_FIRST_LAST[plyri]+', '+str(all_counts[plyri].sum().astype('int')))
    ax[axi,1].set_title(players.DISPLAY_FIRST_LAST[plyri]+', '+str(all_counts[plyri].sum().astype('int')))
    
plt.tight_layout()

In [ ]:
# from highest to lowest (not counting 1)
high = s.sort_values(ascending = False)
high[high.values != 1.0].head()

The pair of players whose correlation coefficients' were highest were Zach Randolph and Salah Mejri with a coefficient of $0.996687$. Now we can visualize their respective shooting patterns next to each other.

In [ ]:
# storing player IDs for the players
randolph = ids_names.loc[ids_names['DISPLAY_FIRST_LAST'] == 'Zach Randolph', 'PERSON_ID'].item()
mejri = ids_names.loc[ids_names['DISPLAY_FIRST_LAST'] == 'Salah Mejri', 'PERSON_ID'].item()

## create figure and axes
fig, ax = plt.subplots(2, 2, figsize=(20,40))

for axi, plyri in enumerate([randolph, mejri]):
    
    h.plot_shotchart(all_counts[plyri], xedges, yedges, ax=ax[axi,0])
    h.plot_shotchart(all_smooth[plyri], xedges, yedges, ax=ax[axi,1])
    
    ax[axi,0].set_title(players.DISPLAY_FIRST_LAST[plyri]+', '+str(all_counts[plyri].sum().astype('int')))
    ax[axi,1].set_title(players.DISPLAY_FIRST_LAST[plyri]+', '+str(all_counts[plyri].sum().astype('int')))
    
plt.tight_layout()

Both seem to be concentrated underneath the basket, but to varying extents. Mejri is basically devoted to the are directly beneath the hoop where in Randolph's case he spends much of his time there but is slightly more spread out about the area than Mejri is. 

### Problem 3

> How would you use the coefficients matrix $H$ from NMF or the correlation matrix $R$ (computed above) to differentiate between types of players? Consider what the coefficients represent, and how you can use them to discriminate player types.

So, we've established that the coefficients matrix $H$ from NMF represents as many modes as we designate the model to represent, and that the correlation matrix $R$ represents the correlation between the individual players. The coefficient matrix $H$ might be better utilized to characterize a 'type' of player, i.e. a 'sharpshooter' who fires from behind the line frequently, someone who dominates the area beneath the hoop, or someone who takes 2-pointers at range. The correlation matrix $R$ better shows how the shooting patterns of one individual relate to another, so it can help with differentiating between individuals, but not between types of players.

> Give your thought process, reasoning for your chosen method, and the results. Do they look reasonable? Do you expect any of the comparison to be similar to any of the figures [here](https://fastbreakdata.com/classifying-the-modern-nba-player-with-machine-learning-539da03bb824)? Why, or why not? Can you verify your intuition?

The utilization of each methods depends on your particular goals in mind. For example, if you want to look at the different shooting patterns of say, all of the shooting guards who played in the 2016-2017 season, then you would create a correlation matrix of those such players to see which players resemble each other, and perhaps visualize their individual patterns to establish similarities and differences. On the other hand, if you want to establish or determine a number of patterns to describe players in the NBA _in general_, then using NMF would be the better route as you can model a certain number of bases. 

Having said that, you can also take the correlation coefficient between a player's individual smoothing statistic and that of a base, to see how individual players measure up to a certain number of pre-determined archetypes. 

Recall that I decided to settle on $8$ bases when using NMF to characterize types of players. It is notable then, that the information on `fastbreakdata.com` (from the link) comes to a similar conclusion--that there are 8 kinds of players in the game, even though there are 5 positions. As mentioned, we can _combine_ the methods of the correlation matrix and the coefficients matrix, to create a heatmap of the players against these 8 types.

In [ ]:
X = pickle.load(open('allpatterns2016-17.pkl', 'rb'))

# 8 components
model8 = skld.NMF(n_components=8, init='nndsvda', max_iter=500, random_state=0)
W = model8.fit_transform(X)
H = model8.components_

Hd = pd.DataFrame(H, columns=all_smooth.keys())
player_coeff = Hd.loc[:,ids.index.values]
player_coeff.columns = ids.DISPLAY_FIRST_LAST

# ensuring the players' coefficients sum to 1
player_coeff.T.sum(1)
player_coeff /= player_coeff.sum(0)
player_coeff2 = player_coeff.T 

#### Players vs Types

The following heatmap compares the 362 players against the 8 'bases' that differentiate between players' shooting patterns. The darker the color, the closer they are to that particular shooting pattern. 

In [ ]:
# heatmap of players vs bases
import seaborn as sns
plt.figure(figsize = (20,100))
sns.heatmap(player_coeff2)

There is a sort of spread distribution between the 8 types, with somewhat of an higher representation of the first type--recall that this is the one which is concentrated directly beneath the hoop. This makes sense considering the fact that the majority of shots are in fact taken from beneath the hoop--this counts dunks, lay-ups, etc. 

When looking at the 5 basic positions, the center, power forward, and small forward all tend to occupy the space inside the 3-point line with greater frequency, but virtually all players spend some amount of time directly under the hoop--so these figures make sense. 

### Problem 4

> Suppose you are in charge of a basketball team. How would you use this information? How would you use what you have learned from analyzing the data, and what other questions would you like to answer with further analysis.

If I was in charge of a basketball team, I would want to create the most diverse team possible, so that each player could cover each others weak points. It's important to note the distinctions not just between players' positions, but also the strengths and weaknesses of players _within_ each position.

Consider for example, the retired player Shaquille O'Neal, who is regarded as one of the greatest players in NBA history. He commanded a very high field goal percentage largely due to his ability to score dunks and hook shots near the basket. Unfortunately he was also notorious for his lack of ability to score free throws. Another example is former MVP Stephen Curry, who is known as a sharpshooter all around the court, from under the rim, to behind the line, to the foul line. His weakness lies in his defensive ability, so his teammates generally cover that responsibility. Both players scarcely played outside of their roles as center and point guard, respectively, making them 'specialists' in that regard.

It is important to note the changes happening within the game itself over time when determining the composition of a team. More recently, the game has evolved to allow for players who are able to shift roles. Draymond Green, who has been deployed as both power forward and a slightly under-sized center, illustrates the versatility that is necessary in the game today. A look at his position in the heatmap above shows that none of the bases describe the majority of his game. As a center, although he may not command the size and power than other centers possess, he does have a playmaking sense, nimbleness and ability to score at distance that most in the position lack.

#### Further Questions

Considering both of these points, I realize that it's actually possible to build a capable team both with 'specialists' like Shaq or Curry, and more versatile players like Green--but having a team full of players who are 'masters of none' may not necessarily be a viable strategy. It would be interesting to try and quantify the relationship between a team's success and the number of specialists vs versatile players it has. 